## Task 1 — Stack (array-backed)

Implement a simple stack with `push`, `pop`, `peek`, `is_empty`, `size`.

In [1]:
class Stack:
    def __init__(self):
        """Array-backed stack; top is the list end."""
        self._data = []

    def push(self, item):
        self._data.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError("Stack underflow")
        return self._data.pop()

    def peek(self):
        if self.is_empty():
            raise IndexError("Empty stack")
        return self._data[-1]

    def is_empty(self):
        return len(self._data) == 0

    def size(self):
        return len(self._data)

# Quick smoke tests
if __name__ == "__main__":
    s = Stack()
    print("Empty?", s.is_empty())
    s.push(10); s.push(20)
    print("After pushes:", s._data)
    print("Peek:", s.peek())
    print("Pop:", s.pop())
    print("Pop:", s.pop())
    try:
        s.pop()
    except IndexError as e:
        print("Expected error on empty pop:", e)

Empty? True
After pushes: [10, 20]
Peek: 20
Pop: 20
Pop: 10
Expected error on empty pop: Stack underflow


## Task 2 — BracketChecker (balanced parentheses)

Uses `Stack` to verify `() [] {}` balance.

In [2]:
class BracketChecker:
    def __init__(self):
        self._stack = Stack()

    def _is_open(self, ch):
        return ch in "([{"

    def _matches(self, open_br, close_br):
        pairs = {')': '(', ']': '[', '}': '{'}
        return pairs.get(close_br) == open_br

    def is_balanced(self, expr: str) -> bool:
        self._stack = Stack()
        for ch in expr:
            if self._is_open(ch):
                self._stack.push(ch)
            elif ch in ")]}":
                if self._stack.is_empty():
                    return False
                top = self._stack.pop()
                if not self._matches(top, ch):
                    return False
        return self._stack.is_empty()

# Quick tests
if __name__ == "__main__":
    bc = BracketChecker()
    tests = ["{[()]}", "([)]", "(((())))", ")("]
    for t in tests:
        print(f"{t} -> Balanced? {bc.is_balanced(t)}")

{[()]} -> Balanced? True
([)] -> Balanced? False
(((()))) -> Balanced? True
)( -> Balanced? False


## Task 3 — Infix → Postfix Converter

Converts single-character-token infix strings to postfix (RPN). Supports ^, *, /, +, -.

In [3]:
class InfixToPostfix:
    def __init__(self):
        self._prec = {'^': 3, '*': 2, '/': 2, '+': 1, '-': 1}
        self._right_assoc = {'^'}
        self._stack = Stack()

    def _is_operand(self, ch):
        return ch.isalnum()

    def convert(self, infix: str) -> str:
        out = []
        self._stack = Stack()
        for ch in infix.replace(" ", ""):
            if self._is_operand(ch):
                out.append(ch)
            elif ch == '(':
                self._stack.push(ch)
            elif ch == ')':
                while not self._stack.is_empty() and self._stack.peek() != '(':
                    out.append(self._stack.pop())
                if self._stack.is_empty():
                    raise ValueError("Mismatched parentheses")
                self._stack.pop()
            else:
                # operator
                while (not self._stack.is_empty()
                       and self._stack.peek() != '('
                       and (self._prec[self._stack.peek()] > self._prec[ch]
                            or (self._prec[self._stack.peek()] == self._prec[ch] and ch not in self._right_assoc))):
                    out.append(self._stack.pop())
                self._stack.push(ch)
        while not self._stack.is_empty():
            top = self._stack.pop()
            if top == '(':
                raise ValueError("Mismatched parentheses")
            out.append(top)
        return "".join(out)

# Quick tests
if __name__ == "__main__":
    conv = InfixToPostfix()
    examples = ["A+B*C", "(A+B)*C", "A^B^C", "A*(B+C*D)"]
    for e in examples:
        print(f"{e} -> {conv.convert(e)}")

A+B*C -> ABC*+
(A+B)*C -> AB+C*
A^B^C -> ABC^^
A*(B+C*D) -> ABCD*+*


## Task 4 — Postfix Evaluator

Simple evaluator for single-digit operands and operators + - * / ^.

In [4]:
class PostfixEvaluator:
    def __init__(self):
        self._stack = Stack()

    def _apply(self, op, b, a):
        if op == '+': return a + b
        if op == '-': return a - b
        if op == '*': return a * b
        if op == '/': return a / b
        if op == '^': return a ** b
        raise ValueError(f"Unknown operator {op}")

    def evaluate(self, postfix: str) -> float:
        self._stack = Stack()
        for ch in postfix.replace(" ", ""):
            if ch.isdigit():
                self._stack.push(float(ch))
            elif ch in "+-*/^":
                if self._stack.size() < 2:
                    raise ValueError("Malformed expression: insufficient operands")
                b = self._stack.pop()
                a = self._stack.pop()
                self._stack.push(self._apply(ch, b, a))
            else:
                raise ValueError(f"Bad token {ch}")
        if self._stack.size() != 1:
            raise ValueError("Malformed expression: leftover values")
        return self._stack.pop()

# Quick tests
if __name__ == "__main__":
    ev = PostfixEvaluator()
    examples = ["432+*", "23+5*", "82/3-"]
    for ex in examples:
        print(f"{ex} -> {ev.evaluate(ex)}")

432+* -> 20.0
23+5* -> 25.0
82/3- -> 1.0


## Task 5 — Linear Search

Simple wrapper class with `find` returning first index or -1.

In [5]:
class LinearSearch:
    def __init__(self, data):
        self.data = list(data)

    def find(self, target):
        for i, x in enumerate(self.data):
            if x == target:
                return i
        return -1

# Quick tests
if __name__ == "__main__":
    ls = LinearSearch([10, 30, 20, 50])
    print("Index of 20:", ls.find(20))
    print("Index of 99:", ls.find(99))

Index of 20: 2
Index of 99: -1
